In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameter Estimation
In this notebook I attempt to approach the designation of parameters anew. The main goal is to make clear the source of all parameters, be it from scientific article, or from a computation. The purpose is to make the research more accessible/understandable, as well as ensuring correctness of parameters following a parameter change.

In [37]:
SHARED_PARAMS = {
    "bbb": [
        ("arteries", "pvs_arteries"),
        ("capillaries", "pvs_capillaries"),
        ("veins", "pvs_veins")
    ],
    "aef": [
        ("pvs_arteries", "ecs"),
        ("pvs_capillaries", "ecs"),
        ("pvs_veins", "ecs")
    ],
    "csf": ["ecs", "pvs_arteries", "pvs_capillaries", "pvs_veins"],
    "blood": ["arteries", "capillaries", "veins"],
    "all": ["ecs", "pvs_arteries", "pvs_capillaries", "pvs_veins", "arteries", "capillaries", "veins"]
}

In [246]:
def retrieve_aliases(param):
    split = param.split("-")
    return [word for word in split[1:] if word in SHARED_PARAMS]


def unpack_shared_parameters(parameters):
    newparameters = {}
    for param, value in parameters.items():
        aliases = retrieve_aliases(param)
        for alias in aliases:
            if type(SHARED_PARAMS[alias][0]) == str:
                newnames = [param.replace(alias, key) for key in SHARED_PARAMS[alias]]
            else:
                newnames = [param.replace(alias, "-".join(key)) for key in SHARED_PARAMS[alias]]
            
            for name in newnames:
                newparameters[name] = value
        
        if len(aliases) == 0:
            newparameters[param] = value
        
    return newparameters

In [247]:
from pint import UnitRegistry
from pint import Quantity
from multirat.parameters import unpack_shared_parameters

ureg = UnitRegistry()

mmHg = ureg.mmHg
Pa = ureg.Pa
mm = ureg.mm
s = ureg.s

In [263]:
base_parameters = {
    # Brain Volume
    "brain_volume": 2450. * mm**3,
    # Pressure boundary parameters
    "pressure_boundary-pvs_arteries": 4.74 * mmHg,
    "pressure_boundary-ecs": 3.74 * mmHg,
    "pressure_boundary-pvs_veins": 3.36 * mmHg,
    "pressure_boundary-arteries": 120. * mmHg,
    "pressure_boundary-veins": 7.0 * mmHg,
    # 
    "diffusion_free_inulin-all": 2.98 * mm**2 / s,
    "tortuosity-all": 1.7,
    # Shared parameters
    "osmotic_pressure-blood": 20. * mmHg,
    "permeability_inulin-bbb": 0.0 * mm / s,  # Does not cross the BBB
}

base_parameters

{'brain_volume': 2450.0 <Unit('millimeter ** 3')>,
 'pressure_boundary-pvs_arteries': 4.74 <Unit('millimeter_Hg')>,
 'pressure_boundary-ecs': 3.74 <Unit('millimeter_Hg')>,
 'pressure_boundary-pvs_veins': 3.36 <Unit('millimeter_Hg')>,
 'pressure_boundary-arteries': 120.0 <Unit('millimeter_Hg')>,
 'pressure_boundary-veins': 7.0 <Unit('millimeter_Hg')>,
 'diffusion_free_inulin-all': 2.98 <Unit('millimeter ** 2 / second')>,
 'tortuosity-all': 1.7,
 'osmotic_pressure-blood': 20.0 <Unit('millimeter_Hg')>,
 'permeability_inulin-bbb': 0.0 <Unit('millimeter / second')>}

In [261]:
def effective_diffusion(D_free, tortuosity):
    return D_free / tortuosity**2


def get_effective_diffusion(p):
    return {
        f"diffusion_eff_inulin-{x}":
            effective_diffusion(p[f"diffusion_free_inulin-{x}"], p[f"tortuosity-{x}"]) for x in SHARED_PARAMS["all"]
    }

In [262]:
unpacked  = unpack_shared_parameters(base_parameters)

derived_parameters = {
    **unpacked,
    **get_effective_diffusion(unpacked)
}
derived_parameters

{'brain_volume': 2450.0 <Unit('millimeter ** 3')>,
 'pressure_boundary-pvs_arteries': 4.74 <Unit('millimeter_Hg')>,
 'pressure_boundary-ecs': 3.74 <Unit('millimeter_Hg')>,
 'pressure_boundary-pvs_veins': 3.36 <Unit('millimeter_Hg')>,
 'pressure_boundary-arteries': 120.0 <Unit('millimeter_Hg')>,
 'pressure_boundary-veins': 7.0 <Unit('millimeter_Hg')>,
 'diffusion_free_inulin-ecs': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-pvs_arteries': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-pvs_capillaries': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-pvs_veins': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-arteries': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-capillaries': 2.98 <Unit('millimeter ** 2 / second')>,
 'diffusion_free_inulin-veins': 2.98 <Unit('millimeter ** 2 / second')>,
 'tortuosity-ecs': 1.7,
 'tortuosity-pvs_arteries': 1.7,
 'tortuosity-pvs_capillaries': 1.7,
 'tortuosity-pv

## Graveyard